<img src="https://s3-sa-east-1.amazonaws.com/preditiva.ai/diversos/preditiva_assinatura.jpg">

# Métodos de Ensemble: *Bagging*

## Importação das bibliotecas

In [1]:
# Módulos de manipulação de dados
import numpy as np
import pandas as pd

# Pacotes gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Pacotes de modelagem
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# Métricas de Desempenho
import preditiva

## Importação da base de dados

In [2]:
df = pd.read_csv('iris_original.csv')
df = df[df['Species'] != 'Iris-setosa']
df['Tipo_Orquidea'] = np.where(df['Species']=='Iris-virginica', 1, 0)
df.head()

Id  comprimento_sepala  largura_sepala  comprimento_petala  \
50  51                 7.0             3.2                 4.7   
51  52                 6.4             3.2                 4.5   
52  53                 6.9             3.1                 4.9   
53  54                 5.5             2.3                 4.0   
54  55                 6.5             2.8                 4.6   

    largura_petala          Species  Tipo_Orquidea  
50             1.4  Iris-versicolor              0  
51             1.5  Iris-versicolor              0  
52             1.5  Iris-versicolor              0  
53             1.3  Iris-versicolor              0  
54             1.5  Iris-versicolor              0

## Preparação dos Dados

### Missing Values

In [3]:
df.isnull().sum()

Id                    0
comprimento_sepala    0
largura_sepala        0
comprimento_petala    0
largura_petala        0
Species               0
Tipo_Orquidea         0
dtype: int64

### Duplicações

In [4]:
df.duplicated().sum()

0

### Tipos de Variáveis

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50 to 149
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  100 non-null    int64  
 1   comprimento_sepala  100 non-null    float64
 2   largura_sepala      100 non-null    float64
 3   comprimento_petala  100 non-null    float64
 4   largura_petala      100 non-null    float64
 5   Species             100 non-null    object 
 6   Tipo_Orquidea       100 non-null    int32  
dtypes: float64(4), int32(1), int64(1), object(1)
memory usage: 5.9+ KB


## Análise Exploratória dos Dados

### Dispersão por Classe

In [ ]:
import matplotlib
matplotlib.use('module://ipykernel.pylab.backend_inline')

sns.pairplot(df, hue='Tipo_Orquidea');

### Relatórios Pandas Profiling e SweetViz

In [ ]:
#perfil_pp, perfil_sv = preditiva.gera_relatorios_aed(df=df,
#                                                     target_feat='Tipo_Orquidea')
sns.reset_defaults()

## Desenvolvimento dos Modelos

### Definição do *Target* e das *Features*

In [6]:
# Target (variável resposta)
y_var = 'Tipo_Orquidea'
y = df[y_var]

# Features (variáveis explicativas)
x_var = ['comprimento_sepala', 'largura_sepala']
x = df[x_var]

In [7]:
y.head()

50    0
51    0
52    0
53    0
54    0
Name: Tipo_Orquidea, dtype: int32

In [8]:
x.head()

comprimento_sepala  largura_sepala
50                 7.0             3.2
51                 6.4             3.2
52                 6.9             3.1
53                 5.5             2.3
54                 6.5             2.8

### Divisão das bases em Treino e Teste

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size=0.30,
                                                    random_state=42)

In [10]:
x_train.shape

(70, 2)

In [11]:
x_test.shape

(30, 2)

### Métodos de Ensemble: *Bagging*

In [12]:
# Inicia o estimador base para o Bagging
modelo_base = LogisticRegression(max_iter=1000)
  
# Quantidade de estimadores
num_estimadores = 100
  
# Classificador de Bagging
modelo_bagging = BaggingClassifier(base_estimator = modelo_base, 
                                   n_estimators = num_estimadores, 
                                   max_samples=0.5,
                                   max_features=0.5,
                                   random_state = 42) 

# Ajuste do Modelo
modelo_bagging.fit(x_train,y_train)

# Métricas de Desempenho
preditiva.calcula_desempenho(modelo_bagging, x_train, y_train, x_test, y_test)

Treino     Teste  Variação
Acurácia   0.728571  0.766667      0.05
AUROC      0.773956  0.812217      0.05
KS         0.511057  0.628959      0.23
Precision  0.725000  0.714286     -0.01
Recall     0.783784  0.769231     -0.02
F1         0.753247  0.740741     -0.02

### Métodos de Ensemble: *Bagging Random Forest*

In [13]:
modelo_rf = RandomForestClassifier(n_estimators=100, 
                                   min_samples_leaf=10, 
                                   max_depth=3,
                                   random_state = 42)
modelo_rf.fit(x_train, y_train)

# Métricas de Desempenho
preditiva.calcula_desempenho(modelo_rf, x_train, y_train, x_test, y_test)

Treino     Teste  Variação
Acurácia   0.771429  0.600000     -0.22
AUROC      0.810401  0.764706     -0.06
KS         0.551188  0.570136      0.03
Precision  0.783784  0.555556     -0.29
Recall     0.783784  0.384615     -0.51
F1         0.783784  0.454545     -0.42

### Modelo individual: Regressão Logística

In [14]:
# Definição do modelo
modelo_rl = LogisticRegression(max_iter=1000)

# Ajuste do Modelo
modelo_rl.fit(x_train, y_train)
  
# Métricas de Desempenho
preditiva.calcula_desempenho(modelo_rl, x_train, y_train, x_test, y_test)

Treino     Teste  Variação
Acurácia   0.742857  0.733333     -0.01
AUROC      0.790336  0.798643      0.01
KS         0.541360  0.570136      0.05
Precision  0.743590  0.666667     -0.10
Recall     0.783784  0.769231     -0.02
F1         0.763158  0.714286     -0.06

### Modelo individual: Árvore de Decisão

In [15]:
# Definição do modelo
modelo_ad = DecisionTreeClassifier(min_samples_leaf=10, 
                                   max_depth=3)

# Ajuste do Modelo
modelo_ad.fit(x_train, y_train)
  
# Métricas de Desempenho
preditiva.calcula_desempenho(modelo_ad, x_train, y_train, x_test, y_test)

Treino     Teste  Variação
Acurácia   0.742857  0.600000     -0.19
AUROC      0.800164  0.701357     -0.12
KS         0.490581  0.470588     -0.04
Precision  0.787879  0.555556     -0.29
Recall     0.702703  0.384615     -0.45
F1         0.742857  0.454545     -0.39